In [1]:
from urllib.request import urlopen
from PIL import Image
import timm

model = timm.create_model('mobilenetv3_small_100.lamb_in1k', pretrained=True)
# model = model.eval()

c:\Users\Justin Clyde\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import torch
# from torchvision.datasets import ImageFolder
# from torchvision.transforms import transforms
# from torch.utils.data import DataLoader

# test_dir = './dataset/test'
# train_dir = './dataset/train'
# val_dir = './dataset/val'

# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# train_ds = ImageFolder(train_dir, transform=transform)
# val_ds = ImageFolder(val_dir, transform=transform)
# test_ds = ImageFolder(test_dir, transform=transform)

# train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
# test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

Preprocess the images on each subfolder


## SKIP THIS

In [3]:
# from PIL import Image, ImageFilter
# import os

# test_dir = './dataset/test'
# train_dir = './dataset/train'
# val_dir = './dataset/validation'

# dir_lst = [test_dir, train_dir, val_dir]

# filters = [ImageFilter.BoxBlur(5), ImageFilter.SHARPEN, ImageFilter.SMOOTH]
# filtername = ["BLUR", "SHARP", "SMOOTH"]

# for dir in dir_lst:
#     for d in os.listdir(dir):
#         for j in os.listdir(f'{dir}/{d}'):
#             with Image.open(f'{dir}/{d}/{j}') as im:
#                 for val in range(3):
#                     im = im.convert("RGB")
#                     new_image = im.filter(filters[val])
#                     new_image.save(f'{dir}/{d}/{j[:-4]}_{filtername[val]}.jpg')


Next step is to create a train and test method

In [4]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset =ImageFolder('./dataset/train', transform=transforms)
val_dataset = ImageFolder('./dataset/validation', transform=transforms)
test_dataset = ImageFolder('./dataset/test', transform=transforms)

train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=12, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False)

In [5]:
import torch

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [6]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for epoch in range(num_epochs):
        model.train()

        running_loss, running_corrects = 0.0, 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)

            _, preds = torch.max(outputs, 1)

            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()*inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        train_loss = running_loss/len(train_loader.dataset)
        train_acc = running_corrects.float()/len(train_loader.dataset)

        model.eval()
        running_loss, running_corrects = 0.0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                _, preds = torch.max(outputs, 1)

                loss = criterion(outputs, labels)
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
        val_loss = running_loss/len(train_loader.dataset)
        val_acc = running_corrects.float()/len(train_loader.dataset)

        print(f'Epoch [{epoch+1}/{num_epochs}], train loss: {train_loss:.4f}, train acc: {train_acc:.4f}, val loss: {val_loss:.4f}, val acc: {val_acc:.4f}')

In [7]:
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
train(model, train_loader, val_loader, criterion, optimizer, 50)

c:\Users\Justin Clyde\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/50], train loss: 4.8587, train acc: 0.0625, val loss: 155904075.1500, val acc: 0.0000
Epoch [2/50], train loss: 2.9345, train acc: 0.2667, val loss: 12977.8104, val acc: 0.0042
Epoch [3/50], train loss: 2.0803, train acc: 0.4000, val loss: 9.2278, val acc: 0.1000
Epoch [4/50], train loss: 1.3189, train acc: 0.6417, val loss: 1.9971, val acc: 0.5750
Epoch [5/50], train loss: 0.7616, train acc: 0.7708, val loss: 1.6596, val acc: 0.6250
Epoch [6/50], train loss: 0.5100, train acc: 0.8708, val loss: 0.7324, val acc: 0.7458
Epoch [7/50], train loss: 0.3487, train acc: 0.8875, val loss: 0.3409, val acc: 0.8792
Epoch [8/50], train loss: 0.3784, train acc: 0.9083, val loss: 0.2436, val acc: 0.9292
Epoch [9/50], train loss: 0.2317, train acc: 0.9250, val loss: 0.0962, val acc: 0.9667
Epoch [10/50], train loss: 0.0805, train acc: 0.9875, val loss: 0.1336, val acc: 0.9375
Epoch [11/50], train loss: 0.1076, train acc: 0.9667, val loss: 0.0059, val acc: 1.0000
Epoch [12/50], train loss: 0.

In [12]:
torch.save(model.state_dict(), f='custom_weights.pt')

In [2]:
# how to load model
# https://github.com/huggingface/pytorch-image-models/discussions/1709

from PIL import Image
import timm
model = timm.create_model(model_name='mobilenetv3_small_100.lamb_in1k',
                          checkpoint_path="custom_weights.pt",
                          pretrained=False) 
model.eval()

img = Image('OP01-001.png')

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)
print(top5_class_indices, top5_probabilities)


c:\Users\Justin Clyde\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: 'module' object is not callable